In [1]:
from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    

load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
def df_to_X_y(df: pd.DataFrame):
    df['days_on_shelf'] = pd.to_datetime(df['date']) - pd.to_datetime(df['release_date'])
    df['days_on_shelf'] = df['days_on_shelf'].dt.days.astype('int16')
    cols = df.columns.tolist()
    cols = cols[:4] + cols[-1:] + cols[4:-1]
    df = df[cols]
    df.pop('date')
    df.pop('release_date')

    df_as_np = df.to_numpy()
    X = df_as_np[:, 4:]
    y = df_as_np[:, :4]
    return X, y

def date_range(start: datetime.date, end: datetime.date, step: datetime.timedelta):
    while start < end:
        yield start
        start += step

In [4]:
df_train = pd.concat([
    pd.read_csv(f'../datasets/{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 9, 30), 
        datetime.date(2020, 1, 6),
        datetime.timedelta(weeks=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 1, 6), 
        datetime.date(2020, 2, 3),
        datetime.timedelta(weeks=1),
    )
])
df_test = pd.concat([
    pd.read_csv(f'../datasets/{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 2, 3), 
        datetime.date(2020, 2, 25),
        datetime.timedelta(weeks=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)
X_test, y_test = df_to_X_y(df_test)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((499639, 22),
 (499639, 4),
 (150896, 22),
 (150896, 4),
 (157826, 22),
 (157826, 4))

In [5]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((22, 1)))
model.add(LSTM(64))
model.add(Dense(8, 'relu'))
model.add(Dense(4, 'linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
Total params: 17,452
Trainable params: 17,452
Non-trainable params: 0
_________________________________________________________________


In [6]:
cp = ModelCheckpoint('../model/model1.keras', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[cp])

Epoch 1/10
15614/15614 [==============================] - 165s 10ms/step - loss: 535.2994 - root_mean_squared_error: 23.1365 - val_loss: 759.3690 - val_root_mean_squared_error: 27.5567
Epoch 2/10
15614/15614 [==============================] - 152s 10ms/step - loss: 428.8335 - root_mean_squared_error: 20.7083 - val_loss: 680.6849 - val_root_mean_squared_error: 26.0899
Epoch 3/10
15614/15614 [==============================] - 151s 10ms/step - loss: 389.3810 - root_mean_squared_error: 19.7327 - val_loss: 650.0763 - val_root_mean_squared_error: 25.4966
Epoch 4/10
15614/15614 [==============================] - 151s 10ms/step - loss: 360.7973 - root_mean_squared_error: 18.9947 - val_loss: 627.9405 - val_root_mean_squared_error: 25.0587
Epoch 5/10
15614/15614 [==============================] - 151s 10ms/step - loss: 337.4073 - root_mean_squared_error: 18.3687 - val_loss: 585.2379 - val_root_mean_squared_error: 24.1917
Epoch 6/10
15614/15614 [==============================] - 157s 10ms/step - 

In [9]:
p = model.predict(X_test)
df = pd.DataFrame({ 
    'predict_view': p[:, 0],
    'view': y_test[:, 0],
    'predict_cart': p[:, 1],
    'cart': y_test[:, 1],
    'predict_remove_from_cart': p[:, 2],
    'remove_from_cart': y_test[:, 2],
    'predict_purchase': p[:, 3],
    'purchase': y_test[:, 3],
})
df.to_csv('test.csv')

4716/4716 [==============================] - 19s 4ms/step
